In [1]:
import time
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision.utils import save_image
import torchvision.transforms.functional as tvtf
import json

import sys; sys.path.append("../src")
from prompts.explanations import *
from cholec import CholecExample, CholecDataset, load_model, items_to_examples
from llms import image_to_base64

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
# llm = load_model("claude-3-5-sonnet-latest")
llm = load_model("gemini-2.5-pro-exp-03-25")
# llm = load_model("gemini-2.0-flash")

In [4]:
dataset = CholecDataset(split="train", image_size=(180,320))
item = dataset[28]
image = item["image"]

/home/antonxue/lib/miniconda3/envs/tfl/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [5]:
prompt = load_cholec_prompt("vanilla") + (image,)

In [6]:
out = llm(prompt)
print(out)

Raw response: (length:  0 )
candidates=[Candidate(content=Content(parts=[Part(video_metadata=None, thought=None, code_execution_result=None, executable_code=None, file_data=None, function_call=None, function_response=None, inline_data=None, text='')], role='model'), citation_metadata=None, finish_message=None, token_count=None, finish_reason=<FinishReason.MAX_TOKENS: 'MAX_TOKENS'>, avg_logprobs=None, grounding_metadata=None, index=0, logprobs_result=None, safety_ratings=None)] create_time=None response_id=None model_version='gemini-2.5-pro-exp-03-25' prompt_feedback=None usage_metadata=GenerateContentResponseUsageMetadata(cache_tokens_details=None, cached_content_token_count=None, candidates_token_count=None, candidates_tokens_details=None, prompt_token_count=6453, prompt_tokens_details=[ModalityTokenCount(modality=<MediaModality.TEXT: 'TEXT'>, token_count=4131), ModalityTokenCount(modality=<MediaModality.IMAGE: 'IMAGE'>, token_count=2322)], thoughts_token_count=4096, tool_use_prompt_t

In [7]:
out

''

In [ ]:
llm(("hello? can you describe this image please:", image))

In [ ]:
dataset = CholecDataset(split="train", image_size=(180,320))
item = dataset[28]
image = item["image"]

# items = []
# for i, item in enumerate(dataset):
#     if i % 5 != 0:
#         continue

#     if (item["gonogo"] == 1).float().mean() > 0.05 and (item["gonogo"] == 2).float().mean() > 0.05:
#         items.append(item)

#     if len(items) >= 10:
#         break

# image = dataset[0]["image"]

In [ ]:
image_to_base64(image)

In [ ]:
tvtf.to_pil_image(image, mode="RGB")

In [ ]:
cholec_vanilla = load_cholec_prompt("vanilla")
cholec_vanilla

In [ ]:
all_prompts = [cholec_vanilla + (i,) for i in range(10)]

In [ ]:
len(all_prompts[0])

In [ ]:
gonogo = item["gonogo"]

In [ ]:
gonogo.shape, gonogo

In [ ]:
tvtf.to_pil_image(gonogo.float(), None)

In [ ]:
tvtf.to_pil_image(image, mode="RGB")

In [ ]:
out = llm(cholec_vanilla + (image,))
print(out)

In [ ]:
dataset = CholecDataset(split="train", image_size=(180,320))
items = []
for i, item in enumerate(dataset):
    if i % 5 != 0:
        continue

    if (item["gonogo"] == 1).float().mean() > 0.05 and (item["gonogo"] == 2).float().mean() > 0.05:
        items.append(item)

    if len(items) >= 10:
        break

In [ ]:
prompt_header = """
You are an expert surgeon with extensive experience in laparoscopic cholecystectomy.
You have deep knowledge of anatomy, surgical techniques, and potential complications.
Your task is to provide an explanation of where it is safe and unsafe to cut based on an image and a list of numbers.
These numbers correspond to a discretization of the image into a 9-high x 16-wide grid.
- The top-left corner is index 0.
- The top-right corner is index 15.
- The bottom-left corner is index 128.
- The bottom-right corner is index 143.

I will give you an example.
"""

In [ ]:
for i, item in enumerate(items):
    image = item["image"]
    safe = (item["gonogo"] == 1).long()
    unsafe = (item["gonogo"] == 2).long()

    a2d = nn.AvgPool2d(kernel_size=20, stride=20, padding=0)
    safe_rough = (a2d(safe.float()).squeeze() > 0.1).long()
    unsafe_rough = (a2d(unsafe.float()).squeeze() > 0.1).long()

    safe_list = safe_rough.view(-1).nonzero().view(-1).tolist()
    unsafe_list = unsafe_rough.view(-1).nonzero().view(-1).tolist()
    
    safe_rough_big = F.interpolate(safe_rough.float().view(1,1,*safe_rough.shape), scale_factor=20).squeeze()
    unsafe_rough_big = F.interpolate(unsafe_rough.float().view(1,1,*safe_rough.shape), scale_factor=20).squeeze()

    explanation = llm((
        prompt_header,
        "Here is the surgery image", image,
        "Here are the grid indices of the safe region: " + str(safe_list),
        "These grid indices correspond to the following mask: ", safe_rough_big,
        "Here are the grid indices of the unsafe region: " + str(unsafe_list),
        "These grid indices correspond to the following mask: ", unsafe_rough_big,
        """Please give me a 300-500 word description of both regions using the image.
        Do not refer to the mask or given indices.
        Your job is to generate an explanation that will be used as part of an LLM prompt.
        That LLM will only be able to see the original surgery image.
        Be concise in your output. Identify only the safe and unsafe regions.
        Do not output broadly general surgical advice.
        """
    ))

    # explanation = ""
    
    path_prefix = f"cholec_fewshot_{i}"
    with open(f"_dump/{path_prefix}_data.json", "w") as f:
        json.dump({
            "id": item["id"],
            "explanation": explanation,
            "safe": safe_list,
            "unsafe": unsafe_list,
        }, f, indent=4)

    save_image(image, f"_dump/{path_prefix}_image.png")
    save_image(safe_rough_big, f"_dump/{path_prefix}_safe.png")
    save_image(unsafe_rough_big, f"_dump/{path_prefix}_unsafe.png")

In [ ]:
print("Safe", safe_list)
print("Unsafe", unsafe_list)
print(explanation)

In [ ]:
plt.clf()
fig, ax = plt.subplots(2,2)
ax[0,0].imshow(safe_rough_big.squeeze().numpy(), cmap="gray")
ax[1,0].imshow(safe.squeeze().numpy(), cmap="gray")
ax[0,1].imshow(unsafe_rough_big.squeeze().numpy(), cmap="gray")
ax[1,1].imshow(unsafe.squeeze().numpy(), cmap="gray")

In [ ]:
(1,) + (5,2)